In [39]:
import os
import requests, bs4
import json
import folium as fm
import pandas as pd
import numpy as np
import geopandas as gpd
import param
import panel as pn
import random
from fastkml.kml import KML
pn.extension(sizing_mode="stretch_width")

In [49]:
wi_df = pd.read_csv('./Static Data/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07


In [53]:
wi_df.head()

,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831
2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466
3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752


In [62]:
wi_df_clean = wi_df[['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','NatWalkInd']]
wi_df_clean.to_csv('Walkability Index.csv',index=False)

In [15]:
request = requests.get('https://geocoding.geo.census.gov/geocoder/geographies/address?street=1250%20Garden%20rd&city=monterey&state=ca&zip=93940&benchmark=2020&vintage=2020&format=JSON')
request.text

'{"result":{"input":{"address":{"zip":"93940","city":"monterey","street":"1250 Garden rd","state":"ca"},"vintage":{"isDefault":true,"id":"2020","vintageName":"Census2020_Census2020","vintageDescription":"Census 2020 Vintage - Census 2020 Benchmark"},"benchmark":{"isDefault":false,"benchmarkDescription":"Public Address Ranges - Census 2020 Benchmark","id":"2020","benchmarkName":"Public_AR_Census2020"}},"addressMatches":[{"tigerLine":{"side":"L","tigerLineId":"635500331"},"geographies":{"State Legislative Districts - Upper":[{"POP100":"","GEOID":"06017","CENTLAT":"+35.9583731","AREAWATER":2489341562,"STATE":"06","BASENAME":"17","OID":212904690192840,"LSADC":"LU","SLDU":"017","FUNCSTAT":"N","INTPTLAT":"+35.9662566","NAME":"State Senate District 17","OBJECTID":925,"CENTLON":"-120.9838511","LSY":"2018","HU100":"","AREALAND":18002094998,"INTPTLON":"-120.9903433","MTFCC":"G5210","LDTYP":"O"}],"States":[{"STATENS":"01779778","POP100":"","GEOID":"06","CENTLAT":"+37.1547578","AREAWATER":20294133

In [47]:
census_block = json.loads(request.text)
census_block['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['STATE']

'06'

In [41]:
street = '298 Cassino Rd'
city = 'Seaside'
state = "CA"
zipcode = '93955'
request = requests.get('https://geocoding.geo.census.gov/geocoder/geographies/address?street=%s&city=%s&state=%s&zip=%s&benchmark=2020&vintage=2020&format=JSON'% 
                       (street, city, state, zipcode))
request.text

'{"result":{"input":{"address":{"zip":"93955","city":"Seaside","street":"298 Cassino Rd","state":"CA"},"vintage":{"isDefault":true,"id":"2020","vintageName":"Census2020_Census2020","vintageDescription":"Census 2020 Vintage - Census 2020 Benchmark"},"benchmark":{"isDefault":false,"benchmarkDescription":"Public Address Ranges - Census 2020 Benchmark","id":"2020","benchmarkName":"Public_AR_Census2020"}},"addressMatches":[{"tigerLine":{"side":"R","tigerLineId":"196821607"},"geographies":{"State Legislative Districts - Upper":[{"POP100":"","GEOID":"06017","CENTLAT":"+35.9583731","AREAWATER":2489341562,"STATE":"06","BASENAME":"17","OID":212904690192840,"LSADC":"LU","SLDU":"017","FUNCSTAT":"N","INTPTLAT":"+35.9662566","NAME":"State Senate District 17","OBJECTID":925,"CENTLON":"-120.9838511","LSY":"2018","HU100":"","AREALAND":18002094998,"INTPTLON":"-120.9903433","MTFCC":"G5210","LDTYP":"O"}],"States":[{"STATENS":"01779778","POP100":"","GEOID":"06","CENTLAT":"+37.1547578","AREAWATER":202941338

In [31]:
data = open('./Static Data/airports.kml').read()

In [16]:
def read_kml(fname='ss.kml'):
    kml = KML()
    kml.from_string(open(fname).read())
    points = dict()
    for feature in kml.features():
        for placemark in feature.features():
            points.update({placemark.name:
                        (placemark.geometry.y, placemark.geometry.x, )})
    return points


fname = './Static Data/airports.kml'
locations = read_kml(fname)

(39.135833, -121.436389)

In [53]:
for _, r in wi[:5].iterrows():
    simple_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = simple_geo.to_json()
    print(simple_geo)

0    POLYGON ((-68983.316 1091325.734, -68981.266 1...
dtype: geometry
0    POLYGON ((-68891.713 1090955.557, -68860.876 1...
dtype: geometry
0    POLYGON ((-68078.320 1091181.799, -68077.077 1...
dtype: geometry
0    POLYGON ((-68978.261 1090638.770, -68976.661 1...
dtype: geometry
0    POLYGON ((-68980.363 1090202.600, -68965.203 1...
dtype: geometry


In [52]:
def get_map(lat=36.598298, long=-121.877012, zoom_start=12):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()
for _, r in wi[:5].iterrows():
    geo = gpd.GeoSeries(r['geometry'])
    geo_j = geo.to_json()
    print(geo_j)
    geo_j = fm.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
    fm.Popup(r['NatWalkInd']).add_to(geo_j)
    geo_j.add_to(map)
    

pn.panel(map, height=400)
map

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-68983.31590000167, 1091325.7336], [-68981.26570000127, 1091373.9231000002], [-68979.82009999827, 1091392.3236999996], [-68975.33170000091, 1091419.2926000003], [-68966.26709999889, 1091460.6206], [-68961.47810000181, 1091478.4370999997], [-68953.46559999883, 1091504.1511000004], [-68939.44579999894, 1091536.9599000001], [-68923.80790000036, 1091574.5543999998], [-68901.34760000184, 1091628.3874000004], [-68897.25329999998, 1091640.63], [-68883.15680000186, 1091635.6102999998], [-68860.9935999997, 1091627.5148999998], [-68835.59200000018, 1091617.2741999999], [-68783.55820000172, 1091599.5729999999], [-68761.941300001, 1091592.9327999996], [-68740.31320000067, 1091587.7430999996], [-68714.5821999982, 1091583.9721999997], [-68667.08650000021, 1091580.5900999997], [-68643.54820000008, 1091581.858], [-68621.21009999886, 1091584.2509000003], [-

In [ ]:
gdb = './Static Data/Natl_WI.gdb/

In [28]:
gdb = './Static Data/Natl_WI.gdb/a00000009.gdbtable'

wi = gpd.read_file(gdb)

# Write to CSV 
#fc.to_csv('/path/to/out_csv.csv')

# Write to GeoJSON
#fc.to_file('./Static Data/Natl_WI.gdb/Walkability.geojson', driver='GeoJSON')

In [51]:
pd.set_option('display.max_columns', None)
wi[:1]

,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,Workers,D2B_E8MIXA,D2A_EPHHM,D3B,D4A,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,geometry
0,481130078254,481130078254,48,113,007825,4,206,"Dallas-Fort Worth, TX-OK",19100,"Dallas-Fort Worth-Arlington, TX",73.595028,0.0,73.595028,73.595028,1202,460.0,423.0,412,0.662091,0.348912,115.981747,362.1,6.0,14.0,15.0,17.0,14.0,3110.36082,297836.08309,"MULTIPOLYGON (((-68983.316 1091325.734, -68981..."
